In [65]:
!pip install langchain_groq langchain_core langchain_community

In [66]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
result =llm.invoke("Who is lord Ram?")
print(result.content)

Lord Rama is a major deity in Hinduism and is considered one of the most important figures in Hindu mythology. He is the seventh avatar (incarnation) of the god Vishnu and is revered as a symbol of righteousness, duty, and virtue.

According to the Hindu epic, the Ramayana, Lord Rama was a king of Ayodhya, a city in ancient India. He was born to King Dasharatha and Queen Kausalya, and was the eldest of four brothers, including Lakshmana, Bharata, and Shatrughna.

The story of Lord Rama is as follows:

* Lord Rama was exiled to the forest for 14 years by his stepmother, Queen Kaikeyi, who wanted her own son, Bharata, to become the king.
* During his exile, Lord Rama's wife, Sita, was abducted by the demon king Ravana, who took her to his kingdom in Lanka (modern-day Sri Lanka).
* Lord Rama, along with his brother Lakshmana and the monkey god Hanuman, embarked on a journey to rescue Sita and defeat Ravana.
* After a long and arduous battle, Lord Rama defeated Ravana and rescued Sita, ret

In [67]:
!pip install pypdf

In [68]:
!pip install chromadb

In [69]:
!pip install sentence_transformers

In [70]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_groq import ChatGroq
def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
  return llm

def create_vector_db():
  loader = DirectoryLoader("/content/DATA", glob = '*.pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain


def main():
  print("Intializing Chatbot.........")
  llm = initialize_llm()

  db_path = "/content/chroma_db"

  if not os.path.exists(db_path):
    vector_db  = create_vector_db()
  else:
    embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  qa_chain = setup_qa_chain(vector_db, llm)

  while True:
    query = input("\nHuman: ")
    if query.lower()  == "exit":
      print("Chatbot: Take Care of yourself, Goodbye!")
      break
    response = qa_chain.run(query)
    print(f"Chatbot: {response}")

if __name__ == "__main__":
  main()

Intializing Chatbot.........

Human: hi


<ipython-input-70-4ed916ac8792>:65: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


Chatbot: Hello! It's nice to meet you. I see you've been reading about mental health and its various definitions. That's a great topic to explore. It's interesting to note how different organizations and researchers define mental health, and how these definitions can impact our understanding of what it means to be mentally healthy.

The World Health Organization's definition of positive mental health, which includes having a sense of purpose, quality relationships, and self-respect, is a great starting point. However, as you mentioned, it's also important to consider the social and physical environments that influence our mental health.

I think it's really important to recognize that mental health is not just an individual issue, but also a collective one. Our relationships, communities, and environments all play a role in shaping our mental wellbeing. And, as you highlighted, adolescents are a crucial group to focus on, as their experiences and attitudes towards mental health can hav

In [71]:
!pip install gradio

In [72]:
import logging
import os
import sys
import signal
import json
import pkg_resources
from datetime import datetime
from typing import Dict, List, Optional
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
import gradio as gr

# Set up logging with more detailed format
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('chatbot.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class UserProfile:
    def __init__(self, user_id: str):
        self.user_id = user_id
        self.preferences = self.load_preferences()
        self.conversation_history = []
        self.mood_tracker = []

    def load_preferences(self) -> Dict:
        try:
            if os.path.exists(f'preferences_{self.user_id}.json'):
                with open(f'preferences_{self.user_id}.json', 'r') as f:
                    return json.load(f)
            return {
                "name": "",
                "preferred_tone": "friendly",
                "response_length": "medium",
                "topics_of_interest": [],
                "triggers_to_avoid": [],
                "preferred_coping_strategies": []
            }
        except Exception as e:
            logger.error(f"Error loading preferences: {str(e)}")
            return {}

    def save_preferences(self):
        try:
            with open(f'preferences_{self.user_id}.json', 'w') as f:
                json.dump(self.preferences, f)
        except Exception as e:
            logger.error(f"Error saving preferences: {str(e)}")

    def update_mood(self, mood: str):
        self.mood_tracker.append({
            "timestamp": datetime.now().isoformat(),
            "mood": mood
        })

class MentalHealthChatbot:
    def __init__(self):
        self.required_packages = [
            "langchain", "gradio", "chromadb",
            "sentence-transformers", "pypdf", "groq"
        ]
        self.active_users: Dict[str, UserProfile] = {}
        self.llm = None
        self.vector_db = None
        self.qa_chain = None
        self.is_running = True

        # Set up signal handlers
        signal.signal(signal.SIGINT, self.handle_shutdown)
        signal.signal(signal.SIGTERM, self.handle_shutdown)

    def handle_shutdown(self, signum, frame):
        """Gracefully handle shutdown signals"""
        logger.info("Shutdown signal received. Cleaning up...")
        self.is_running = False
        self.cleanup()
        sys.exit(0)

    def cleanup(self):
        """Perform cleanup operations"""
        logger.info("Performing cleanup...")
        # Save all user profiles
        for user_id, profile in self.active_users.items():
            profile.save_preferences()
        if self.vector_db:
            self.vector_db.persist()

    def check_dependencies(self):
        for package in self.required_packages:
            try:
                pkg_resources.require(package)
            except pkg_resources.DistributionNotFound:
                logger.error(f"Required package {package} is not installed")
                sys.exit(1)

    def initialize_llm(self):
        try:
            logger.info("Initializing LLM...")
            self.llm = ChatGroq(
                temperature=0.3,
                groq_api_key="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",
                model_name="llama-3.3-70b-versatile"
            )
        except Exception as e:
            logger.error(f"Error initializing LLM: {str(e)}")

    def create_vector_db(self):
        try:
            logger.info("Creating vector database...")
            loader = DirectoryLoader("/content/DATA", glob='*.pdf', loader_cls=PyPDFLoader)
            documents = loader.load()
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
            texts = text_splitter.split_documents(documents)
            embeddings = HuggingFaceBgeEmbeddings(
                model_name='sentence-transformers/all-MiniLM-L6-v2'
            )
            self.vector_db = Chroma.from_documents(
                texts,
                embeddings,
                persist_directory='./chroma_db'
            )
            self.vector_db.persist()
        except Exception as e:
            logger.error(f"Error creating vector database: {str(e)}")

    def get_personalized_prompt(self, user_profile: UserProfile) -> str:
        """Generate a personalized prompt template based on user preferences"""
        name = user_profile.preferences.get("name", "")
        tone = user_profile.preferences.get("preferred_tone", "friendly")
        length = user_profile.preferences.get("response_length", "medium")

        base_prompt = f"""You are a {tone} mental health chatbot{' speaking to ' + name if name else ''}.
        Recent mood: {user_profile.mood_tracker[-1]['mood'] if user_profile.mood_tracker else 'Unknown'}

        Guidelines:
        - Keep responses {length} in length
        - Avoid triggering topics: {', '.join(user_profile.preferences.get('triggers_to_avoid', []))}
        - Preferred coping strategies: {', '.join(user_profile.preferences.get('preferred_coping_strategies', []))}

        Context information: {{context}}

        User: {{question}}
        Chatbot: """

        return base_prompt

    def setup_qa_chain(self):
        try:
            logger.info("Setting up QA chain...")
            retriever = self.vector_db.as_retriever(search_kwargs={"k": 2})

            # Using a default prompt template initially
            default_prompt = """You are a friendly and concise mental health chatbot.
            Context information: {context}
            User: {question}
            Chatbot: """

            PROMPT = PromptTemplate(
                template=default_prompt,
                input_variables=['context', 'question']
            )

            self.qa_chain = RetrievalQA.from_chain_type(
                llm=self.llm,
                chain_type="stuff",
                retriever=retriever,
                chain_type_kwargs={
                    "prompt": PROMPT,
                    "verbose": True
                }
            )
        except Exception as e:
            logger.error(f"Error setting up QA chain: {str(e)}")

    def get_or_create_user_profile(self, user_id: str) -> UserProfile:
        """Get existing user profile or create new one"""
        if user_id not in self.active_users:
            self.active_users[user_id] = UserProfile(user_id)
        return self.active_users[user_id]

    def chatbot_response(self, user_input: str, user_id: str) -> tuple[str, List]:
        user_profile = self.get_or_create_user_profile(user_id)

        try:
            if not user_input.strip():
                return "Please provide a valid input.", user_profile.conversation_history

            # Handle mood tracking
            mood_keywords = {
                "happy": ["happy", "joy", "excited", "great"],
                "sad": ["sad", "down", "depressed", "upset"],
                "anxious": ["anxious", "worried", "nervous", "stressed"],
                "angry": ["angry", "mad", "frustrated", "annoyed"]
            }

            for mood, keywords in mood_keywords.items():
                if any(keyword in user_input.lower() for keyword in keywords):
                    user_profile.update_mood(mood)

            # Update QA chain with personalized prompt
            personalized_prompt = self.get_personalized_prompt(user_profile)
            self.qa_chain.combine_documents_chain.llm_chain.prompt = PromptTemplate(
                template=personalized_prompt,
                input_variables=['context', 'question']
            )

            response = self.qa_chain.run(user_input)
            user_profile.conversation_history.append((user_input, response))
            return response, user_profile.conversation_history

        except Exception as e:
            error_msg = f"Error: {str(e)}"
            logger.error(error_msg)
            return error_msg, user_profile.conversation_history

    def create_gradio_interface(self):
        with gr.Blocks(theme='Respair/Shiki@1.2.1') as app:
            gr.Markdown("#  Personalized Mental Health Chatbot🧠")

            with gr.Tab("Chat"):
                chatbot = gr.Chatbot(height=400)
                msg = gr.Textbox(label="Message", placeholder="Type your message here...")
                user_id = gr.Textbox(label="User ID", placeholder="Enter your user ID")

            with gr.Tab("Preferences"):
                name = gr.Textbox(label="Your Name")
                tone = gr.Dropdown(
                    choices=["friendly", "professional", "casual", "supportive"],
                    label="Preferred Tone"
                )
                response_length = gr.Dropdown(
                    choices=["short", "medium", "long"],
                    label="Response Length"
                )
                triggers = gr.TextArea(label="Current Feelings (comma-separated)")
                strategies = gr.TextArea(label="Preferred Coping Strategies (comma-separated)")
                save_btn = gr.Button("Save Preferences")

            def save_preferences(user_id_val, name_val, tone_val, length_val, triggers_val, strategies_val):
                if not user_id_val:
                    return "Please enter a user ID"

                user_profile = self.get_or_create_user_profile(user_id_val)
                user_profile.preferences.update({
                    "name": name_val,
                    "preferred_tone": tone_val,
                    "response_length": length_val,
                    "triggers_to_avoid": [t.strip() for t in triggers_val.split(",") if t.strip()],
                    "preferred_coping_strategies": [s.strip() for s in strategies_val.split(",") if s.strip()]
                })
                user_profile.save_preferences()
                return "Preferences saved successfully!"

            def respond(message, chat_history, user_id_val):
                if not user_id_val:
                    return "Please enter a user ID first", chat_history
                bot_message, updated_history = self.chatbot_response(message, user_id_val)
                chat_history.append((message, bot_message))
                return "", chat_history

            msg.submit(respond, [msg, chatbot, user_id], [msg, chatbot])
            save_btn.click(
                save_preferences,
                [user_id, name, tone, response_length, triggers, strategies],
                gr.Textbox(label="Status")
            )

            gr.Markdown(
                "*For urgent issues, please seek help from licensed mental health professionals.*"
            )

        return app

    def run(self):
        self.check_dependencies()
        self.initialize_llm()

        if not self.llm:
            logger.error("Failed to initialize LLM")
            return

        db_path = "./chroma_db"

        # Load or create vector database
        if not os.path.exists(db_path):
            self.create_vector_db()
        else:
            try:
                embeddings = HuggingFaceBgeEmbeddings(
                    model_name='sentence-transformers/all-MiniLM-L6-v2'
                )
                self.vector_db = Chroma(
                    persist_directory=db_path,
                    embedding_function=embeddings
                )
            except Exception as e:
                logger.error(f"Error loading existing vector database: {str(e)}")
                return

        if not self.vector_db:
            logger.error("Failed to initialize vector database")
            return

        self.setup_qa_chain()
        if not self.qa_chain:
            logger.error("Failed to setup QA chain")
            return

        app = self.create_gradio_interface()
        app.launch()

if __name__ == "__main__":
    chatbot = MentalHealthChatbot()
    chatbot.run()

/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:282: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9d0a89faec96e688f7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
